#### Imports and dataset loading

In [2]:
import numpy as np
import pickle
from scipy.special import softmax
from utils import utils_vectorize as uv

with open('datasets/def2def_samples.pkl', 'rb') as f: 
    def2def_samples, def2def_gold_scores = pickle.load(f)

with open('datasets/def2def_adjusted_samples.pkl', 'rb') as f: 
    def2def_adjusted_samples, def2def_adjusted_scores = pickle.load(f)

with open('datasets/def2def250_adjusted_samples.pkl', 'rb') as f: 
    def2def250_samples, def2def250_scores = pickle.load(f)

#### Model functions

In [3]:
def fact_to_vector(fact:tuple, stop_words=False, punct_marks=False, embed_model='w2v', mu='ratio', weights=(1., 1., 2.)):
    # 1. A sequential composition into each element of triplet
    v_subj = uv.icds_vectorize(fact[0], stop_words=stop_words, punct_marks=punct_marks, embed_model=embed_model, mu=mu) * weights[0] #* 1.2 #* 1.1 
    v_rel = uv.icds_vectorize(fact[1], stop_words=stop_words, punct_marks=punct_marks, embed_model=embed_model, mu=mu) * weights[1] #* 0.6 #* .45 
    v_obj = uv.icds_vectorize(fact[2], stop_words=stop_words, punct_marks=punct_marks, embed_model=embed_model, mu=mu) * weights[2] #* 1.5 #* 1.6 
    # 2. A full composition on whole triplet
    #v_fact = v_subj + v_rel + v_obj
    v_subj_rel = uv.icds_composition(v_subj, v_rel)
    v_subj_obj = uv.icds_composition(v_subj, v_obj)
    v_rel_obj = uv.icds_composition(v_rel, v_obj)
    v_fact = uv.icds_composition(v_subj_rel, v_obj) 
    #v_fact = uv.icds_composition(v_subj_obj, v_rel) 
    #v_fact = uv.icds_composition(v_subj, v_rel_obj)   
    return(v_fact, v_subj, v_rel, v_obj)

# Self-attention
def kgtxt_to_selfatt_vectors(txt_kg, stop_words=False, punct_marks=False, beta=1.2, embed_model='w2v', mu='ratio', weights=(1., 1., 2.)):
    n_facts = len(txt_kg)
    weight_mtrx = np.empty((n_facts, n_facts))
    v_kg = []
    for txt_fact in txt_kg:
        v_fact, _, _, _ = fact_to_vector(txt_fact, stop_words=stop_words, punct_marks=punct_marks, embed_model=embed_model, mu=mu, weights=weights)
        v_kg.append(v_fact)
    # 1. Compute fact-wise similarity (self-attention scores)
    for idx in range(n_facts):
        norm0 = np.max([1.e-125, np.linalg.norm(v_kg[idx])])
        for jdx in range(n_facts):
            norm1 = np.max([1.e-125, np.linalg.norm(v_kg[jdx])])
            norms_ratio = max(1.e-125, np.min([norm0, norm1])/np.max([norm0, norm1])) 
            # Joint information content = IC(x,y) = IC(x) + IC(y) - <x,y> 
            #weight_mtrx[idx][jdx] = norm0**2 + norm1**2 - v_kg[idx] @ v_kg[jdx] #uv.cos_sim(v_kg[idx], v_kg[jdx])) 
            weight_mtrx[idx][jdx] = (norms_ratio ** 2) * uv.cos_sim(v_kg[idx], v_kg[jdx]) 
    # 2. Softmax/normalizing (self-att weights)
    #norm_weight_mtrx = softmax(weight_mtrx, axis=1) 
    #norm_weight_mtrx = np.apply_along_axis(np.tanh, axis=1, arr=weight_mtrx) 
    #norm_weight_mtrx = np.apply_along_axis(uv.sigmoid, axis=1, arr=weight_mtrx) 
    norm_weight_mtrx = weight_mtrx
    # 3. Context vectors (new contextual embeddings)
    self_att_mtrx = norm_weight_mtrx @ np.array(v_kg) 
    #print(self_att_mtrx)
    return(self_att_mtrx, np.array(v_kg))

In [4]:
# Functions to get similarity between two KGs
# Combinations: composition = (sum | ICDS); sim = (cos | ICM | euclid | dot prod); sents_sim: (mean | median | bidir | bertscore)
def pair_sim(kg_pair, self_att=True, stop_words=True, punct_marks=False, beta=1.2, embed_model='w2v', mu='ratio', weights=(1., 1., 2.)):  # List of tuples of 3 strings
    sims = []
    kg0 = kg_pair[0]
    kg1 = kg_pair[1]
    n_facts_kg0 = len(kg0)
    n_facts_kg1 = len(kg1)
    n_sims = n_facts_kg0 * n_facts_kg1
    sim_mtrx = np.empty((n_facts_kg0, n_facts_kg1))
    if self_att:
        self_att_mtrx0, _ = kgtxt_to_selfatt_vectors(kg0, stop_words=stop_words, punct_marks=punct_marks, beta=beta, embed_model=embed_model, mu=mu, weights=weights)
        self_att_mtrx1, _ = kgtxt_to_selfatt_vectors(kg1, stop_words=stop_words, punct_marks=punct_marks, beta=beta, embed_model=embed_model, mu=mu, weights=weights)
        for idx in range(n_facts_kg0):
            norm0 = np.max([1.e-125, np.linalg.norm(self_att_mtrx0[idx])])
            for jdx in range(n_facts_kg1):
                norm1 = np.max([1.e-125, np.linalg.norm(self_att_mtrx1[jdx])])
                norms_weight = np.max(max(1.e-125, np.min([norm0, norm1])/np.max([norm0, norm1])))
                ic_01 = (abs(norm0 + norm1 - self_att_mtrx0[idx] @ self_att_mtrx1[jdx])) 
                sim_mtrx[idx][jdx] =  max(0, uv.cos_sim(self_att_mtrx0[idx], self_att_mtrx1[jdx])) 
                #sim_mtrx[idx][jdx] = uv.icm_sim(self_att_mtrx0[idx], self_att_mtrx1[jdx], beta=beta) 
                #sim_mtrx[idx][jdx] = np.linalg.norm(self_att_mtrx0[idx] - self_att_mtrx1[jdx])         
                #sim_mtrx[idx][jdx] = self_att_mtrx0[idx] @ self_att_mtrx1[jdx].T 
    else:           # No self-attention
        for idx, fact0 in enumerate(kg0):
            fact0_vector, _, _, _ = fact_to_vector(fact0, stop_words=stop_words, punct_marks=punct_marks, embed_model=embed_model, mu=mu, weights=weights) 
            norm0 = np.linalg.norm(fact0_vector)
            for jdx, fact1 in enumerate(kg1):
                fact1_vector, _, _, _ = fact_to_vector(fact1, stop_words=stop_words, punct_marks=punct_marks, embed_model=embed_model, mu=mu, weights=weights) 
                norm1 = np.linalg.norm(fact1_vector)
                sim_mtrx[idx][jdx] = max(0, uv.cos_sim(fact0_vector, fact1_vector)) 
                #sim_mtrx[idx][jdx] = uv.icm_sim(fact0_vector, fact1_vector, beta=1.4) 
    #print(sim_mtrx)
    #sents_sim = np.mean(sim_mtrx) 
    sents_sim = uv.bidir_avgmax_sim(sim_mtrx, stdst='mean') 
    #sents_sim = uv.bertscore(sim_mtrx)
    return(sents_sim)

# Correlation with true in a dataset of KG pairs
def ds_sents_sim(ds, true_scores, self_att=True, stop_words=True, punct_marks=False, beta=1.2, embed_model='w2v', mu='ratio', weights=(1., 1., 2.)):
    sims = []
    for pair in ds:
        sims.append(pair_sim(pair, self_att=self_att, stop_words=stop_words, punct_marks=punct_marks, beta=beta, embed_model=embed_model, mu=mu, weights=weights))
    correlation = np.corrcoef(sims, np.array(true_scores))[0][1]
    return(correlation, np.array(sims))

### Tests

With original DEF2DEF

In [5]:
# Test combinations: embedding = ('w2v', 'glove'); mu = (0, 1, 'ratio')
samples = def2def_samples
scores = def2def_gold_scores
corr, sims = ds_sents_sim(samples, scores, self_att=False,  
                        stop_words=True, punct_marks=True, beta=1.5, embed_model='w2v', mu=1, weights=(1.2, 0.6, 1.5))   
print('True scores min, max, mean and std:',np.min(scores), np.max(scores), np.mean(scores), np.std(scores))
print('Sim scores min, max, mean and std:', np.min(sims*5), np.max(sims*5), np.mean(sims*5), np.std(sims*5))
print('Corr. for MSRpar:', corr, '\n')

True scores min, max, mean and std: 0 50 25.029067824924823 12.310826756521202
Sim scores min, max, mean and std: 0.0 5.00000019868215 1.4328392425011043 0.7484405117155343
Corr. for MSRpar: 0.512111440065369 



In [6]:
# Best results | This file: ok 20250318
paper_ref =     [0.53]  # W2V + BEST STR + F.INF + COS
no_self_att =   [0.5121]  # s_words ok | sim_mtrx=bidir mean | SRO ws: 1.2 0.6 1.5  | mu=1 | v_fact=subj_obj·rel
self_att =      [0.5141]  # s_words ok | sim_mtrx=bidir mean | SRO ws: 1.1 0.45 1.6 | mu=1 | v_fact=subj_obj·rel | sa=no_sa

With adjusted DEF2DEF

In [7]:
# Test combinations: embedding = ('w2v', 'glove'); mu = (0, 1, 'ratio')
samples = def2def_adjusted_samples
scores = def2def_adjusted_scores
corr, sims = ds_sents_sim(samples, scores, self_att=True,  
                        stop_words=True, punct_marks=True, beta=1.5, embed_model='w2v', mu=1, weights=(1.2, 0.7, 1.65))   
print('True scores min, max, mean and std:',np.min(scores), np.max(scores), np.mean(scores), np.std(scores))
print('Sim scores min, max, mean and std:', np.min(sims*5), np.max(sims*5), np.mean(sims*5), np.std(sims*5))
print('Corr. for MSRpar:', corr, '\n')

True scores min, max, mean and std: 0 50 24.631877729257642 12.341513586651518
Sim scores min, max, mean and std: 0.0 5.0 1.6909885097834403 0.8561488768689401
Corr. for MSRpar: 0.5843654125589703 



In [8]:
# Best results with adjusted DEF2DEF | This file: ok 20250327
no_self_att =   [0.581]  # s_words ok | sim_mtrx=bidir mean | SRO ws: 1.1 0.6 1.3 | mu=1 | v_fact=subj_rel·obj 
self_att =      [0.584]  # s_words ok | sim_mtrx=bidir mean | SRO ws: 1.2 0.7 1.65 | mu=1 | v_fact=subj_obj·rel 

With DEF2DEF_250

In [9]:
# Test combinations: embedding = ('w2v', 'glove'); mu = (0, 1, 'ratio')
samples = def2def250_samples
scores = def2def250_scores
corr, sims = ds_sents_sim(samples, scores, self_att=True,  
                        stop_words=True, punct_marks=True, beta=1.5, embed_model='w2v', mu=1, weights=(1., 0.2, 1.5))   
print('True scores min, max, mean and std:',np.min(scores), np.max(scores), np.mean(scores), np.std(scores))
print('Sim scores min, max, mean and std:', np.min(sims*5), np.max(sims*5), np.mean(sims*5), np.std(sims*5))
print('Corr. for MSRpar:', corr, '\n')

True scores min, max, mean and std: 1 49 23.904 12.394465861827205
Sim scores min, max, mean and std: 0.023298098055828866 5.0 1.6291712005364851 0.8461793445885769
Corr. for MSRpar: 0.6065150766396528 



In [10]:
# Best results with adjusted DEF2DEF and reduction to _250_ samples | This option: ok 20250329
no_self_att =   [0.601]  # s_words ok | sim_mtrx=bidir mean | SRO ws: 1.1 0.4 1.5 | mu=1 | v_fact=subj_rel·obj 
self_att =      [0.607]  # s_words ok | sim_mtrx=bidir mean | SRO ws: 1.0 0.2 1.5 | mu=1 | v_fact=subj_rel·obj 